In [1]:
%load_ext lab_black
%cd ../../..

/home/shim/shim/dev/log-analytics


In [2]:
import argparse
import math
import multiprocessing
import sys
from datetime import datetime
from pathlib import Path
from pprint import pformat

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_optimizer
import yaml
from easydict import EasyDict
from pytorch_transformers import DistilBertForSequenceClassification, DistilBertTokenizer

from datasets.dataset_ver6 import DatasetGeneratorVer6
from utils import SAM, AverageMeter, CustomLogger, FocalLoss, seed_everything

from main_ver6 import MyTrainer
from collections import defaultdict
import matplotlib.pyplot as plt
import random
from sklearn.metrics import f1_score, classification_report
from tqdm import tqdm

In [3]:
postfix = "distilbert-base-uncased-focal-AdamW-lr2e-05-ver6-os100_3"
outdir = Path("results/distilbert-base-uncased-ver6")
fold = 3

In [4]:
with open("config/distilbert-base-uncased-ver6.yaml", "r") as f:
    C = EasyDict(yaml.load(f, yaml.FullLoader))
    C.result_dir = Path(C.result_dir)
    C.dataset.dir = Path(C.dataset.dir)
    seed_everything(C.seed, deterministic=False)

In [5]:
C

{'model': {'name': 'distilbert-base-uncased'},
 'comment': None,
 'result_dir': PosixPath('results/distilbert-base-uncased-ver6'),
 'debug': False,
 'seed': 20210425,
 'ver': 6,
 'train': {'SAM': False,
  'folds': [1],
  'checkpoints': [None],
  'loss': {'name': 'focal',
   'params': {'gamma': 2.0, 's': 45.0, 'm': 0.1, 'crit': 'focal'}},
  'optimizer': {'name': 'AdamW'},
  'finetune': {'do': True, 'step1_epochs': 2, 'step2_epochs': 3},
  'max_epochs': 6,
  'lr': 2e-05,
  'scheduler': {'name': 'ReduceLROnPlateau',
   'params': {'factor': 0.5, 'patience': 3, 'verbose': True}}},
 'dataset': {'dir': PosixPath('data/ver6'),
  'batch_size': 35,
  'num_workers': 8,
  'oversampling': True,
  'oversampling_scale': 100}}

In [6]:
trainer = MyTrainer(C, fold, f"results/distilbert-base-uncased-ver6/{postfix}.pth")

Load pretrained results/distilbert-base-uncased-ver6/distilbert-base-uncased-focal-AdamW-lr2e-05-ver6-os100_3.pth


In [7]:
model = trainer.model
model.eval()
torch.set_grad_enabled(False)

In [8]:
activation = []


def hook(model, input, output):
    activation.append(output.detach().cpu())

In [9]:
model.pre_classifier.register_forward_hook(hook)

In [10]:
train_dl = trainer.dsgen.train_only()

# 덱만들기 (Train)

In [12]:
activation = []
deck = {"feat": [], "otext": [], "tlevel": [], "fclevel": []}
with tqdm(total=len(train_dl.dataset), ncols=100, file=sys.stdout) as t:
    for text, tlevel, otext in train_dl:
        fclevel = model(text.cuda(non_blocking=True))[0].argmax(dim=1).cpu()
        deck["fclevel"].append(fclevel)
        deck["tlevel"].append(tlevel)
        deck["otext"].extend(otext)
        t.update(text.size(0))

100%|██████████████████████████████████████████████████████| 421079/421079 [20:54<00:00, 335.54it/s]


In [13]:
deck["feat"] = torch.cat(activation)
deck["tlevel"] = torch.cat(deck["tlevel"])
deck["fclevel"] = torch.cat(deck["fclevel"])

In [14]:
deck["tlevel"].shape, deck["fclevel"].shape

(torch.Size([421079]), torch.Size([421079]))

In [15]:
deck["feat"].shape

torch.Size([421079, 768])

In [19]:
torch.save(deck, outdir / f"{postfix}-deck1.pth")

# Valid Lv7에 대한 결과 분석

In [20]:
ds_lv7 = trainer.dsgen.valid_lv7().dataset

In [25]:
text, otext = ds_lv7[0]
activation = []
print(model(text[None].cuda())[0].cpu())

tensor([[-6.0883,  5.0010, -3.5641, -3.7608, -7.7354, -4.5796, -7.5599]])


In [26]:
dists, indices = torch.norm(deck["feat"] - activation[0][None], p=None, dim=1).topk(4, largest=False)
dists, indices, deck["tlevel"][indices]

(tensor([[24.0634, 24.9287, 28.6150, 30.3823]]),
 tensor([[487,  19,  79, 606]]),
 tensor([[1, 0, 0, 1]]))

In [27]:
text, otext = ds_lv7[1]
activation = []
print(model(text[None].cuda())[0].cpu())

tensor([[-2.1250, -3.0019, -6.5012,  1.4310, -6.4430, -1.8724, -6.9261]])


In [28]:
dists, indices = torch.norm(deck["feat"] - activation[0][None], p=None, dim=1).topk(4, largest=False)
dists, indices, deck["tlevel"][indices]

(tensor([[25.0888, 27.9535, 41.0431, 41.8491]]),
 tensor([[ 19, 719, 601, 170]]),
 tensor([[0, 0, 0, 0]]))

In [29]:
text, otext = ds_lv7[2]
activation = []
print(model(text[None].cuda())[0].cpu())

tensor([[ 3.9369, -4.8182, -4.7900, -2.6410, -7.5933, -3.9964, -5.5910]])


In [30]:
dists, indices = torch.norm(deck["feat"] - activation[0][None], p=None, dim=1).topk(4, largest=False)
dists, indices, deck["tlevel"][indices]

(tensor([[24.9473, 28.7714, 33.3087, 37.8109]]),
 tensor([[ 19, 549, 474, 317]]),
 tensor([[0, 1, 1, 0]]))

# 덱만들기 (Submission)

메모리가 적으면 여기서 reload 권장

In [11]:
ds_test = trainer.dsgen.test()

In [12]:
activation = []
deck = {"feat": [], "otext": [], "fclevel": [], "ids": []}
with tqdm(total=len(ds_test), ncols=100, file=sys.stdout) as t:
    for i in range(len(ds_test)):
        text, otext, ids = ds_test[i]
        fclevel = model(text[None].cuda(non_blocking=True))[0].argmax(dim=1).cpu()
        deck["fclevel"].append(fclevel)
        deck["otext"].append(otext)
        deck["ids"].append(ids)
        t.update()

100%|██████████████████████████████████████████████████| 1095951/1095951 [2:00:58<00:00, 150.98it/s]


In [13]:
deck["feat"] = torch.stack(activation)
deck["fclevel"] = torch.stack(deck["fclevel"])

In [14]:
deck["feat"] = deck["feat"][:, 0, :]
deck["fclevel"] = deck["fclevel"][:, 0]

In [15]:
deck["feat"].shape, deck["fclevel"].shape

(torch.Size([1095951, 768]), torch.Size([1095951]))

In [16]:
torch.save(deck, outdir / f"{postfix}-deck2.pth")